In [15]:
import pandas as pd
import os

# --- CONFIGURATION ---
FILE_PATH = "data_sensible/24 12 Mais.Droit STATS 2024.xlsx"
OUTPUT_SQL = "alimentation_supabase.sql"
MOIS_SHEETS = ['Jan', 'Fev', 'Mar', 'Avr', 'Mai', 'Juin', 'Juil', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc']

def format_val(val):
    """Formate les valeurs pour SQL (gère les NULL et les apostrophes)"""
    if pd.isna(val) or str(val).lower() in ['nan', 'none', 'null', '']:
        return "NULL"
    if isinstance(val, (int, float)):
        return str(val)
    # CORRECTION SYNTAXE : Utilisation de guillemets simples à l'intérieur pour éviter le SyntaxError
    val_clean = str(val).replace("'", "''")
    return f"'{val_clean}'"

def generer_sql():
    print(f"--- GÉNÉRATION DU FICHIER SQL ---")
    
    if not os.path.exists(FILE_PATH):
        print(f" [ERREUR] Fichier Excel introuvable : {FILE_PATH}")
        return

    with open(OUTPUT_SQL, "w", encoding="utf-8") as f:
        # 1. Nettoyage initial (cite: image_ae9546.png)
        f.write("-- NETTOYAGE DES TABLES\n")
        f.write("TRUNCATE TABLE entretien, demande, solution RESTART IDENTITY CASCADE;\n\n")

        all_demandes = []
        all_solutions = []

        for idx, sheet_name in enumerate(MOIS_SHEETS, 1):
            try:
                df = pd.read_excel(FILE_PATH, sheet_name=sheet_name)
                
                # Nettoyage des données
                df['N°'] = pd.to_numeric(df['N°'], errors='coerce')
                df = df.dropna(subset=['N°'])
                df['N°'] = df['N°'].astype(int)
                df = df.drop_duplicates(subset=['N°'], keep='first')
                df = df.sort_values(by='N°')

                if df.empty: continue

                f.write(f"-- DONNÉES DE L'ONGLET : {sheet_name}\n")
                
                # 2. GÉNÉRATION INSERT POUR 'entretien'
                f.write("INSERT INTO entretien (num, date_ent, mode, duree, sexe, age, vient_pr, sit_fam, enfant, modele_fam, profession, ress, origine, commune, partenaire) VALUES\n")
                
                rows_ent = []
                for _, row in df.iterrows():
                    date_val = f"'2024-{idx:02d}-01'"
                    
                    line = [
                        str(row['N°']),
                        date_val,
                        format_val(row.get('Mode')),
                        format_val(row.get('Durée')),
                        format_val(row.get('Sexe')),
                        format_val(row.get('Age')),
                        format_val(row.get('Vient pr')),
                        format_val(row.get('Sit° Fam')),
                        format_val(row.get('Enfts')),
                        format_val(row.get('Modèle fam.')),
                        format_val(row.get('Prof°')),
                        format_val(row.get('Ress. 1')),
                        format_val(row.get('Origine')),
                        format_val(row.get('Domicile')),
                        format_val(row.get('Partenaire'))
                    ]
                    rows_ent.append(f"({', '.join(line)})")
                
                f.write(",\n".join(rows_ent) + ";\n\n")

                # 3. COLLECTE DES DEMANDES ET SOLUTIONS
                for _, row in df.iterrows():
                    for i, col in enumerate(['Dem.1', 'Dem.2', 'Dem.3'], 1):
                        val = row.get(col)
                        if pd.notna(val) and str(val).strip() not in ["", "nan"]:
                            all_demandes.append(f"({int(row['N°'])}, {i}, {format_val(val)})")
                    
                    for i, col in enumerate(['Sol.1', 'Sol.2', 'Sol.3'], 1):
                        val = row.get(col)
                        if pd.notna(val) and str(val).strip() not in ["", "nan"]:
                            all_solutions.append(f"({int(row['N°'])}, {i}, {format_val(val)})")

                print(f" [OK] {sheet_name} traité.")

            except Exception as e:
                print(f" [ERREUR] {sheet_name} : {e}")

        # 4. INSERTION FINALE (BULK INSERT)
        def write_bulk(name, data, columns):
            if not data: return
            f.write(f"-- INSERTION DANS {name.upper()}\n")
            # On insère par paquets pour éviter de saturer l'éditeur SQL
            for i in range(0, len(data), 500):
                batch = data[i:i+500]
                f.write(f"INSERT INTO {name} {columns} VALUES\n")
                f.write(",\n".join(batch) + ";\n")
            f.write("\n")

        write_bulk("demande", all_demandes, "(num, pos, nature)")
        write_bulk("solution", all_solutions, "(num, pos, nature)")

        # 5. Synchronisation de la séquence (cite: image_aeefd2.png)
        f.write("-- MISE À JOUR DE LA SÉQUENCE D'ID\n")
        f.write("SELECT setval('entretien_num_seq', (SELECT MAX(num) FROM entretien));\n")

    print(f"\n--- TERMINÉ ---")
    print(f"Fichier créé : {os.path.abspath(OUTPUT_SQL)}")

if __name__ == "__main__":
    generer_sql()

--- GÉNÉRATION DU FICHIER SQL ---
 [OK] Jan traité.
 [OK] Fev traité.
 [OK] Mar traité.
 [OK] Avr traité.
 [OK] Mai traité.
 [OK] Juin traité.
 [OK] Juil traité.
 [OK] Aoû traité.
 [OK] Sep traité.
 [OK] Oct traité.
 [OK] Nov traité.
 [OK] Déc traité.

--- TERMINÉ ---
Fichier créé : C:\Users\gabin\Desktop\Cours\projet dudu\Analyse-et-conception-d-un-outil-d-cisionnel-pour-MD\alimentation_supabase.sql
